# 通过净利润的思想来挑选股票

In [1]:
import pandas as pd
import numpy as np
import time

import sys
sys.path.append("../utils/")
import token_util
pro = token_util.set_token()

*获取股票列表*

In [2]:
stock_list = pd.read_csv("../data_pulled/stock_date_delta_priceNone.csv")
stock_list.head()

,ts_code,name,total_mv
0,000002.SZ,万科A,3.009761e+07
1,000012.SZ,南玻A,1.370715e+06
2,000016.SZ,深康佳A,2.090097e+06
3,000021.SZ,深科技,2.551164e+06
4,000027.SZ,深圳能源,2.128932e+06


In [3]:
stock_list.shape

(749, 3)

### 价值股
1.ROE>10%</br>
2.流动比率>150%，速动比率>100%</br>
3.净利润逐年增长且大于10%</br>
4.公司财务四年内都符合以上三个要求

*获取数据*

In [4]:
# 获取 roe
def get_roe(ts_code, start, end):
    for i in range(start, end):
                
        try:
            
            ret = pro.fina_indicator(ts_code=ts_code, period='201'+str(i)+'1231', fields='ts_code,end_date,roe')
            rets = pd.concat([rets, ret], ignore_index=True) if i != 4 else ret
            
        except:
            time.sleep(3)
    
    return rets.drop_duplicates()[:5].reset_index(drop=True)
    

# 获取 income
def get_income(ts_code, start, end):
    for i in range(start, end):
        
        try:
            
            ret = pro.income(ts_code=ts_code, period='201'+str(i)+'1231', fields='ts_code,end_date,n_income')
            rets = pd.concat([rets, ret], ignore_index=True) if i != 4 else ret
    
        except:
            time.sleep(3)
            
    return rets.drop_duplicates()[:5].reset_index(drop=True)

# 获取资产负债信息
def get_balancesheet(ts_code, start, end):
    for i in range(start, end):
        
        try:
        
            ret = pro.balancesheet(ts_code=ts_code, period='201'+str(i)+'1231', fields='ts_code,end_date,inventories,\
                                    total_cur_assets, total_cur_liab')
            ret["liquidity_ratio"] = ret["total_cur_assets"]/ret["total_cur_liab"] * 100
            ret["quick_ratio"] = (ret["total_cur_assets"]-ret["inventories"])/ret["total_cur_liab"] * 100
            rets = pd.concat([rets, ret], ignore_index=True) if i != 4 else ret
                            
        except:
            time.sleep(3)
            
    rets = rets.drop(labels=["inventories","total_cur_assets","total_cur_liab"], axis=1)
    
    return rets.drop_duplicates()[:5].reset_index(drop=True)

*获取处理后的数据*

In [5]:
def get_data(stock_code, start, end):
    data = get_roe(stock_code, start, end)
    ret1 = get_income(stock_code, start, end)
    ret2 = get_balancesheet(stock_code, start, end)

    data["n_income"] = ret1["n_income"]
    data["liquidity_ratio"] = ret2["liquidity_ratio"]
    data["quick_ratio"] = ret2["quick_ratio"]
    
    return data

*进行筛选*

In [10]:
candidate_list = []

for stock_code in stock_list["ts_code"][617:]:
    
    data = get_data(stock_code, 4, 9)
    pass_data = True
    
    for i in range(1, len(data)):
        roe = data.iloc[i]["roe"]
        n_income_last = data.iloc[i-1]["n_income"]
        n_income_cur = data.iloc[i]["n_income"]
        liquidity_ratio = data.iloc[i]["liquidity_ratio"]
        quick_ratio = data.iloc[i]["quick_ratio"]

        if roe > 10 and liquidity_ratio > 150 and quick_ratio > 100 and\
            n_income_cur > 0 and n_income_cur > n_income_last and\
            (n_income_cur - n_income_last) / n_income_last >= 0.15:
            continue
        else:
            pass_data = False
            break
            
    if pass_data is True:
        candidate_list.append(stock_code)   

In [11]:
# stock_code = "000917.SZ"
# start, end = 4, 9

# data = get_roe(stock_code, start, end)
# ret1 = get_income(stock_code, start, end)
# ret2 = get_balancesheet(stock_code, start, end)

# data["n_income"] = ret1["n_income"]
# data["liquidity_ratio"] = ret2["liquidity_ratio"]
# data["quick_ratio"] = ret2["quick_ratio"]

In [12]:
len(candidate_list)

6

In [13]:
candidate_list

['603228.SH', '603338.SH', '603589.SH', '603658.SH', '603866.SH', '603899.SH']

In [9]:
stock_list[(stock_list["ts_code"] == stock_code)]

,ts_code,name,total_mv
617,600901.SH,江苏租赁,1.523191e+06
